In [1]:
import pandas as pd 
import fastai

df = pd.read_csv("downloads/40k_balanced_pm_acl.csv").sample(frac=1)
df = df.dropna(subset=["labels"])
df["text_nomask"] = df["text"] #backup
print(len(df))
df_test = df[:300]
df = df[300:]

fastai.__version__, len(df)

42140


('2.5.3', 41840)

In [6]:
df["text_length"]= df["text"].map(lambda t: len(t.split(" ")))

In [8]:
df = df[df["text_length"] > 7]


In [9]:
from octoflow.core import replace_outof_vocab_words, vocab30k

df["text"] = df["text"].map(lambda t: replace_outof_vocab_words(t, vocab30k))
df_test["text"] = df_test["text"].map(lambda t: replace_outof_vocab_words(t,vocab30k))

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df["text"] = df["text"].astype(str)
df["labels"] = df["labels"].astype(int)

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
df = df[(df["labels"] == 1) | (df["labels"] == 0)]
df["lab"] = df["labels"] #worst fastai bug in history

In [12]:
from fastai.text.all import *
dls = TextDataLoaders.from_df(df, text_col='text', label_col='lab')

dls.show_batch()

learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, FBeta(beta=1)])
learn.fine_tune(2, 1e-2)
learn.show_results()


interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(3)  
interp.plot_confusion_matrix()

learn.predict("We discuss the implications of a more 'civic-minded journalism.")

In [14]:
learn.fine_tune(2, 1e-2)

epoch     train_loss  valid_loss  accuracy  fbeta_score  time    
0         0.039256    0.034981    0.987027  0.987857     00:22     
epoch     train_loss  valid_loss  accuracy  fbeta_score  time    
0         0.037926    0.032206    0.988016  0.988824     00:45     
1         0.030831    0.029084    0.988386  0.989168     00:46     


In [15]:
learn.export('oct15_40k.pkl')
learn.save("oc15_40k")

Path('models/oc15_40k.pth')

## Sanity Check: Tough Cases
0 = non-problem
1 = problem

In [16]:
learn.predict("Anaphylaxis is a life-threatening emergency of which reliable epidemiological data are lacking")

('1', TensorText(1), TensorText([0.0353, 0.9647]))

In [17]:
predicts = df_test["text"].map(learn.predict)

In [18]:
df_test["predicted"] = [int(p[0]) for p in predicts]
df_test["confidence_0"] =  [round(p[2][0].item(),2) for p in predicts]
df_test["confidence_1"] =  [round(p[2][1].item(),2) for p in predicts]

In [19]:
df_test.to_csv("downloads/oct15_40k_test300.csv")

In [20]:
print("accuracy in test set:", len(df_test[df_test["predicted"] == df_test["labels"]])/len(df_test))

accuracy in test set: 0.9833333333333333


In [21]:
predicts2 = df_test["text_nomask"].map(learn.predict)

In [22]:
df_test["predicted2"] = [int(p[0]) for p in predicts2]
print("accuracy in test set with no masking:", len(df_test[df_test["predicted2"] == df_test["labels"]])/len(df_test))

accuracy in test set with no masking: 0.9866666666666667


That's very weird ... masking doesn't effect predictions